In [4]:

"""Dependencies
-customtkinter
-tkinter
-toml
-tomlkit
"""
#Gui for communicating stuff
# Credits to: https://github.com/TomSchimansky/CustomTkinter/blob/master/examples/complex_example.py
# Import necessary libraries
import customtkinter
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import executor_func as exeFunc
import threading
import tkinter as tk

# Define lists for error mitigation techniques and overhead
ErrorMitImps = ["Zero-Noise Extrapolation", "Digital Dynamical Decoupling",
                ] #  "Probabilistic Error Cancellation", "Readout error Mitigation",
Overhead = ["Circuit Depth", "Number of Single Qubit Gates", "Number of Two Qubit Gates","Time"]

# loadingText = [
#     r"_____ ______   ___  _________  ___  ________",
#     r"|\   _ \  _  \|\  \|\___   ___\\  \|\   __  \ ",
#     r"\ \  \\\__\ \  \ \  \|___ \  \_\ \  \ \  \|\  \ ",
#     r" \ \  \\|__| \  \ \  \   \ \  \ \ \  \ \  \\\  \ ",
#     r"  \ \  \    \ \  \ \  \   \ \  \ \ \  \ \  \\\  \ ",
#     r"   \ \__\    \ \__\ \__\   \ \__\ \ \__\ \_____  \ ",
#     r"    \|__|     \|__|\|__|    \|__|  \|__|\|___| \__\ ",
#     r"                                               \|__|"
# ]
loadingText = [
    ".-----------------------------------------------------.",
    "|  _____ ______   ___  _________  ___  ________       |",
    "| |\\   _ \\  _   \\|\\  \\|\\___   ___\\\\  \\|\\   __  \\      |",
    "| \\ \\  \\\\\\__\\ \\  \\ \\  \\|___ \\  \\_\\ \\  \\ \\  \\|\\  \\     |",
    "|  \\ \\  \\\\|__| \\  \\ \\  \\   \\ \\  \\ \\ \\  \\ \\  \\\\\\  \\    |",
    "|   \\ \\  \\    \\ \\  \\ \\  \\   \\ \\  \\ \\ \\  \\ \\  \\\\\\  \\   |",
    "|    \\ \\__\\    \\ \\__\\ \\__\\   \\ \\__\\ \\ \\__\\ \\_____  \\  |",
    "|     \\|__|     \\|__|\\|__|    \\|__|  \\|__|\\|___| \\__\\ |",
    "|                                               \\|__| |",
    "'-----------------------------------------------------'"
]
# Calculate the maximum length of the strings
max_length = max(len(line) for line in loadingText)

# Pad each string with spaces at the end to match the maximum length
loadingText = [line.ljust(max_length) for line in loadingText]

# # Join the lines with "\n" to form the ASCII art
# loading_text = "\n".join(loadingText)

# print(loading_text)

# Set appearance mode and default color theme for customtkinter
customtkinter.set_appearance_mode("System")  # Modes: "System" (standard), "Dark", "Light"
customtkinter.set_default_color_theme("green")  # Themes: "blue" (standard), "green", "dark-blue"

# Define the GUI application class
class appWindow(customtkinter.CTk):
    def __init__(self):
        super().__init__()

        # Configure window
        self.title("Mitigation Implementation")
        self.width = self.winfo_screenwidth()
        self.height = self.winfo_screenheight()
        window_width = int(self.winfo_screenwidth())  # 80% of screen width
        window_height = int(self.winfo_screenheight() * 0.9)  # 80% of screen height
        # Set the geometry of the window
        self.geometry(f"{window_width}x{window_height}+{-10}+{-1}")
        # Configure grid layout (4x4)
        self.grid_columnconfigure(1, weight=1)
        self.grid_columnconfigure((2, 3), weight=0)
        self.grid_rowconfigure((0, 1, 2), weight=1)

        # Create tabview
        self.tabview = customtkinter.CTkTabview(self, width=10, height=500, anchor="w")
        self.tabview.grid(row=0, column=1, padx=(20, 0), pady=(20, 0), sticky="nsew")
        self.tabview.add("User Info")
        self.tabview.add("Graph")
        # self.tabview.add("Tab 3")
        self.tabview.tab("Graph").grid_columnconfigure((0,1,2), weight=1)  # configure grid of individual tabs
        self.tabview.tab("User Info").grid_columnconfigure((0,1,2), weight=1)

        # Create label for Tab 2
        self.label_tab_2 = customtkinter.CTkLabel(self.tabview.tab("User Info"), text="",
                                                  font=customtkinter.CTkFont(size=40, weight="bold"))
        self.label_tab_2.grid(row=0, column=0, padx=20, pady=20)

        # create main entry and button
        self.entry = customtkinter.CTkEntry(self.label_tab_2, placeholder_text="Azure ID:")
        self.entry.grid(row=3, column=1, columnspan=2, padx=(20, 0), pady=(20, 20), sticky="nsew")

        self.main_button_1 = customtkinter.CTkButton(master=self.label_tab_2, text="Enter", fg_color="transparent", border_width=2, text_color=("gray10", "#DCE4EE"))
        self.main_button_1.grid(row=3, column=3, padx=(20, 20), pady=(20, 20), sticky="nsew")



        # Load image for Tab 3
        # image = customtkinter.CTkImage(light_image=Image.open(r"C:\Users\ben10\Downloads\potato.jpg"), size=(30, 30))
        # self.Image = customtkinter.CTkLabel(self.tabview.tab("Tab 3")) #, image=image

        # Create button for updating the graph
        self.sidebar_button_1 = customtkinter.CTkButton(self.tabview.tab("Graph"), text="Make Graph",
                                                        command=self.update_window)
        self.sidebar_button_1.grid(row=3, column=2, padx=20, pady=10)


        # Create sidebar frame with widgets
        self.sidebar_frame = customtkinter.CTkFrame(self, width=140, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(4, weight=1)
        self.logo_label = customtkinter.CTkLabel(self.sidebar_frame, text="MITIQ",
                                                  font=customtkinter.CTkFont(size=40, weight="bold"))
        self.logo_label.configure(text_color='yellow')
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))



        self.defaultMode = customtkinter.CTkSwitch(self.sidebar_frame, text="Default Mode")
        self.defaultMode.grid(row=4, column=0, padx=20, pady=(10, 0))

        # Widgets for appearance mode and UI scaling
        self.appearance_mode_label = customtkinter.CTkLabel(self.sidebar_frame, text="Appearance Mode:", anchor="w")
        self.appearance_mode_label.grid(row=5, column=0, padx=20, pady=(10, 0))
        self.appearance_mode_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["Light", "Dark", "System"],
                                                                       command=self.change_appearance_mode_event)
        self.appearance_mode_optionemenu.grid(row=6, column=0, padx=20, pady=(10, 10))
        self.scaling_label = customtkinter.CTkLabel(self.sidebar_frame, text="UI Scaling:", anchor="w")
        self.scaling_label.grid(row=7, column=0, padx=20, pady=(10, 0))
        self.scaling_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame,
                                                               values=["80%", "90%", "100%", "110%", "120%"],
                                                               command=self.change_scaling_event)
        self.scaling_optionemenu.grid(row=8, column=0, padx=20, pady=(10, 20))

        # Create scrollable frames for error implementations and overhead
        framesInGraphTab = 0
        framesInGraphTab, self.scrollable_frame_switches_implements = self.create_scrollable_frame(self.tabview.tab("Graph"), "Error Implementations", ErrorMitImps,framesInGraphTab)
        framesInGraphTab, self.scrollable_frame_switches_overhead = self.create_scrollable_frame(self.tabview.tab("Graph"), "Overhead", Overhead, framesInGraphTab)


        # Set default values
        self.appearance_mode_optionemenu.set("Dark")
        self.scaling_optionemenu.set("100%")
        self.scrollable_frame_switches_implements[0].select()
        self.scrollable_frame_switches_overhead[0].select()
        self.defaultMode.select()
        # self.textbox.insert("0.0", "AER User ID")


        # Plotting section
        self.plotSize = (self.width * .004, self.height * .005)
        
        
        # Loading text ascii image
        self.placeholder_label = customtkinter.CTkLabel(self.tabview.tab("Graph"), text="Make Graph when ready", anchor="center", wraplength=0,text_color="yellow")
        self.placeholder_label.grid(row=0, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")

        self.update()

    # Method to update the graph
    def update_window(self):
        plt.close()
        
        switchesImp = self.get_selected_switches(self.scrollable_frame_switches_implements)
        switchesOver = self.get_selected_switches(self.scrollable_frame_switches_overhead)
        print(len(switchesImp))
        if len(switchesImp) == 0:
            # Ask to choose an implementation
            imp_label = customtkinter.CTkLabel(self.tabview.tab("Graph"), text="Please select an Implementation", anchor="center",text_color="yellow")
            imp_label.grid(row=0, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")
        elif len(switchesImp) == 1:
                print("Selected implementations is: " +str(switchesImp[0]))
                self.title = str(switchesImp[0])
        else:
             print("Need to add multi implementation graph option")
             return

        if len(switchesImp) == 0:
            # Ask to choose an implementation
            placeholder_label = customtkinter.CTkLabel(self.tabview.tab("Graph"), text="Please select an Overhead", anchor="center",text_color="yellow")
            placeholder_label.grid(row=0, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")
        elif len(switchesImp) == 1:
                print("Selected overhead is: " +str(switchesOver))
                self.plot_it_async(tech=str(switchesImp[0]), overheads = switchesOver, plotSize=self.plotSize, path_to_metadata='metadata.toml')
                # # Create a CTkLabel widget and add display text
                # if switchesImp[0] == "Zero-Noise Extrapolation":
                #     name = "ZNE"
                # elif switchesImp[0] == "Digital Dynamical Decoupling":
                #     name = "DDD"
                # elif switchesImp[0] == "Readout error Mitigation":
                #     name = "REM"
                # elif switchesImp[0] == "Probabilistic Error Cancellation":
                #     name = "PEC"
                # summary = "summary_"+name+".toml"
                # text = exeFunc.appBackend().getMetadataString(summary)
                # label = customtkinter.CTkLabel(self.tabview.tab("Graph"), text=text)

                # # Place the label in the window using grid
                # label.grid(row=2, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")
        else:
             print("Need to add multi-overhead graph option")
             return

    # Method to handle appearance mode change event
    def change_appearance_mode_event(self, new_appearance_mode: str):
        customtkinter.set_appearance_mode(new_appearance_mode)

    # Method to handle UI scaling change event
    def change_scaling_event(self, new_scaling: str):
        new_scaling_float = int(new_scaling.replace("%", "")) / 100
        customtkinter.set_widget_scaling(new_scaling_float)

    # Method to create scrollable frame
    def create_scrollable_frame(self, parent, label_text, items, count):
        scrollable_frame = customtkinter.CTkScrollableFrame(parent, label_text=label_text, fg_color="purple")
        scrollable_frame.grid(row=0, column=count, padx=20, pady=20, columnspan=1, sticky="nsew")
        scrollable_frame.grid_columnconfigure(0, weight=1)
        scrollable_frame_switches = []
        for i, item in enumerate(items):
            switch = customtkinter.CTkSwitch(master=scrollable_frame, text=item)
            switch.grid(row=i, column=0, padx=10, pady=20)
            scrollable_frame_switches.append(switch)
        count = count +1
        return count, scrollable_frame_switches
        
            
    def get_selected_switches(self, switches):
        selected_switches = []
        for switch in switches:
            if switch.get():
                selected_switches.append(switch.cget("text"))
        return selected_switches
    
    # Method to plot asynchronously
    def plot_it_async(self, tech,overheads, plotSize, path_to_metadata):
        def update_label(count):
            if count <= len(loadingText):
                # Display loading text as ASCII art
                ascii_art = [loadingText[i] for i in range(count)]
                loading_text = "\n".join(ascii_art)
                self.placeholder_label.configure(text=loading_text,font=("Courier", 12))
                # Schedule the next update after a delay
                self.after(500, update_label, count + 1 )
            elif count == len(loadingText)+1:
                # Display loading text as ASCII art
                ascii_art = [loadingText[i] for i in range(count-1)]
                loading_text = "\n".join(ascii_art)
                loading_text = loading_text +"\nRunning Error Mitigation..."
                self.placeholder_label.configure(text=loading_text,font=("Courier", 12))
                # Schedule the next update after a delay
                self.after(500, update_label, count + 1 )

        
        def plot_async():
            # Start updating the label text
            self.placeholder_label = customtkinter.CTkLabel(self.tabview.tab("Graph"), anchor="center", text_color="yellow", font=("Courier", 12))
            self.placeholder_label.grid(row=0, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")
            update_label(0)

            # Perform any necessary processing before plotting
            fig, ax = exeFunc.appBackend().start(tech=tech, plotSize=plotSize,overheads=overheads, path_to_metadata=path_to_metadata, gui = True)
            # After the plotting is done, update the GUI with the results
            self.update_graph_async(fig, ax)

        self.placeholder_label.configure(text="Running Error Mitigation...")
        thread = threading.Thread(target=plot_async)
        thread.start()


    # Add the update_graph_async method
    def update_graph_async(self, fig, ax):
        switchesImp = self.get_selected_switches(self.scrollable_frame_switches_implements)
        switchesOver = self.get_selected_switches(self.scrollable_frame_switches_overhead)
        ax.set_xlabel(r'$N_{qubits}$')
        ax.set_ylabel(f"{switchesOver[0]}")
        ax.set_title(f"{switchesImp[0]}'s {switchesOver[0]} vs. Qubit Number ")
        ax.legend(fontsize=str(int(.0070*self.width)), loc ="upper left")
        
        canvas = FigureCanvasTkAgg(fig, self.tabview.tab("Graph"))
        canvas.draw()
        canvas.get_tk_widget().grid(row=0, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew", columnspan=1, rowspan=1)
        self.sidebar_button_1.configure(text="Update Graph")
        
        self.update()

            
if __name__ == "__main__":
    # Create the default root window
    root = tk.Tk()
    # Create the application window
    app = appWindow()
    # Run the application
    app.mainloop()

1
Selected implementations is: Zero-Noise Extrapolation
Selected overhead is: ['Circuit Depth']
tech = Zero-Noise Extrapolation
0.27300500869750977
0.9999999999999983
0.31888580322265625
0.0
0.4853043556213379
0.9999999999999983
0.4546315670013428
0.9999999999999983
Summarizing metadata information.........
[0.25893235 0.30185366 0.4587307  0.43058968]
1
Selected implementations is: Digital Dynamical Decoupling
Selected overhead is: ['Time']
tech = Digital Dynamical Decoupling
0.09277153015136719
1.0
0.06809473037719727
1.0
0.10531067848205566
0.0


Exception ignored in: <function Image.__del__ at 0x0000018861C7A340>
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 4105, in __del__
    self.tk.call('image', 'delete', self.name)
RuntimeError: main thread is not in main loop
Exception ignored in: <function Image.__del__ at 0x0000018861C7A340>
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 4105, in __del__
    self.tk.call('image', 'delete', self.name)
RuntimeError: main thread is not in main loop
Exception ignored in: <function Image.__del__ at 0x0000018861C7A340>
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 4105, in __del__
    self.tk.call('image', 'delete', self.na

11.496715307235718
0.0
Summarizing metadata information.........
[ 0.0820663   0.04804873  0.07973552 11.47303295]


In [2]:
# # Change jupyter of the executor_func_gui into a python script
# # executor_func_gui must be a script to be able to import
!jupyter nbconvert --to python executor_func.ipynb 

[NbConvertApp] Converting notebook executor_func.ipynb to python
[NbConvertApp] Writing 29847 bytes to executor_func.py


In [ ]:
# # Example of importing the appBackend class from executor_func_gui 
# from executor_func_gui import appBackend

# # Example of using appBackend class
# appBackend().get_circ_bs_lists(4)